# Ver jugar a un modelo de Super Mario preentrenado

https://github.com/jorgecasase/mariobrosplay-windows-gym-RL

Este notebook permite cargar y observar cómo juega un modelo de Super Mario preentrenado. Utiliza un entorno personalizado basado en OpenAI Gym, junto con varias optimizaciones específicas para juegos retro.

## Requisitos
1. Haber creado el entorno virutal con conda

2. Un modelo preentrenado debe estar disponible en el archivo `trained_mario.chkpt`. (celda 9)
   - Si no tienes un modelo preentrenado, puedes entrenarlo siguiendo el tutorial en el siguiente enlace:
     [Mario RL con PyTorch - Entrenamiento en 400 episodios](https://github.com/pedroconcejero/deep_learning_2024/blob/main/mario_RL_pytorch_tutorial_400_episodes_save_every_1e4.ipynb)
     - o puedes usar el de github
2. Asegúrate de tener instaladas todas las dependencias necesarias, incluidas `gym`, `gym_super_mario_bros`, y `torch`.

Una vez cumplidos los requisitos, puedes cargar el modelo y observar su rendimiento en el entorno.

Nota: este notebook está adaptado para ser ejecutado en windows nativo, no funciona en colab ni en macos.

## Importación de modulos

In [1]:
# Importación de módulos necesarios
import random, datetime
from pathlib import Path

# Importación de librerías para Gym y Super Mario
import gym
import gym_super_mario_bros
from gym.wrappers import FrameStack, GrayScaleObservation, TransformObservation
from nes_py.wrappers import JoypadSpace

# Importación de módulos personalizados
from metrics import MetricLogger
from agent import Mario
from wrappers import ResizeObservation, SkipFrame

## Creación del entorno

In [2]:
# Creación del entorno base de Super Mario
env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')

# Configuración del espacio de acciones (Joypad)
env = JoypadSpace(
    env,
    [['right'],  # Acción 1: caminar a la derecha
    ['right', 'A']]  # Acción 2: caminar a la derecha y saltar
)

## Aplicación de wrappers

In [3]:
# Aplicación de wrappers para modificar el entorno
env = SkipFrame(env, skip=4)  # Saltar frames para mejorar el rendimiento
env = GrayScaleObservation(env, keep_dim=False)  # Convertir observaciones a escala de grises
env = ResizeObservation(env, shape=84)  # Redimensionar observaciones a 84x84 píxeles
env = TransformObservation(env, f=lambda x: x / 255.)  # Normalizar observaciones
env = FrameStack(env, num_stack=4)  # Apilar 4 frames consecutivos

# Reiniciar el entorno para comenzar
env.reset()

## Cargar modelo preentrenado

In [4]:
# Configuración del directorio de guardado para puntos de control y métricas
save_dir = Path('checkpoints') / datetime.datetime.now().strftime('%Y-%m-%dT%H-%M-%S')
save_dir.mkdir(parents=True)  # Crear el directorio si no existe

# Inicialización del agente Mario con un punto de control preentrenado **AQUI CARGAS TU MODELO**
checkpoint = Path('mario_net_13.chkpt')
mario = Mario(state_dim=(4, 84, 84), action_dim=env.action_space.n, save_dir=save_dir, checkpoint=checkpoint)

# Fijar la tasa de exploración al mínimo
mario.exploration_rate = mario.exploration_rate_min

# Inicialización del registro de métricas
logger = MetricLogger(save_dir)

C:\Users\Jorge Casas\Downloads\MadMario-master\agent.py:168: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckp = torch.load(load_path, map_location=('cuda' if self.use_cuda 

Loading model at mario_net_13.chkpt with exploration rate 0.9680224458943157


## Verlo jugar

In [5]:
# Número de episodios de entrenamiento
episodes = 100

# Bucle de entrenamiento
for e in range(episodes):

    # Reiniciar el entorno y obtener el estado inicial
    state = env.reset()

    while True:
        # Renderizar el entorno
        env.render()

        # Elegir una acción con el agente
        action = mario.act(state)

        # Ejecutar la acción en el entorno
        next_state, reward, done, info = env.step(action)

        # Almacenar la transición en la memoria del agente
        mario.cache(state, next_state, action, reward, done)

        # Registrar métricas del paso
        logger.log_step(reward, None, None)

        # Actualizar el estado actual
        state = next_state

        # Salir del bucle si el juego termina o Mario alcanza la meta
        if done or info['flag_get']:
            break

    # Registrar métricas del episodio
    logger.log_episode()

    # Guardar métricas cada 20 episodios
    if e % 20 == 0:
        logger.record(
            episode=e,
            epsilon=mario.exploration_rate,
            step=mario.curr_step
        )

C:\Users\Jorge Casas\Downloads\MadMario-master\agent.py:57: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  state = torch.FloatTensor(state).cuda() if self.use_cuda else torch.FloatTensor(state)
C:\miniconda3\envs\mario\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in scalar subtract
  return (self.ram[0x86] - self.ram[0x071c]) % 256


Episode 0 - Step 40 - Epsilon 0.1 - Mean Reward 231.0 - Mean Length 40.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 4.896 - Time 2024-12-12T01:30:45


C:\miniconda3\envs\mario\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in scalar subtract
  return (self.ram[0x86] - self.ram[0x071c]) % 256


Episode 20 - Step 7729 - Epsilon 0.1 - Mean Reward 308.0 - Mean Length 368.048 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 159.371 - Time 2024-12-12T01:33:25


C:\miniconda3\envs\mario\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in scalar subtract
  return (self.ram[0x86] - self.ram[0x071c]) % 256


...

Episode 60 - Step 20163 - Epsilon 0.1 - Mean Reward 337.115 - Mean Length 330.541 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 133.02 - Time 2024-12-12T01:37:39


C:\miniconda3\envs\mario\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in scalar subtract
  return (self.ram[0x86] - self.ram[0x071c]) % 256


Episode 80 - Step 27940 - Epsilon 0.1 - Mean Reward 343.864 - Mean Length 344.938 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 162.873 - Time 2024-12-12T01:40:22


C:\miniconda3\envs\mario\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in scalar subtract
  return (self.ram[0x86] - self.ram[0x071c]) % 256


<Figure size 640x480 with 0 Axes>